# Code for Login, navigating to Exports Page and Parsing the HTML

In [ ]:
import requests
import os
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Define the URL
url = 'https://app.dimensions.ai/exports'

# Login page URL
login_url = 'https://app.dimensions.ai/login' # Update with actual login page URL

# Path to your Chromedriver executable
path_to_chromedriver = '/Users/sachinsrivastava/Downloads/chromedriver'

# Create the driver instance
driver = webdriver.Chrome(executable_path=path_to_chromedriver)

# Open the login page
driver.get(login_url)

# Wait for login to complete
time.sleep(60)

# Navigate to the data page
driver.get(url)

# Add delay for page to load
time.sleep(5)

# Fetch page source
html = driver.page_source

# Create soup object
soup = BeautifulSoup(html, 'lxml')

# Find table rows
rows = soup.find_all('tr', {'data-bt': 'table-row'})

# Prepare list to hold all row data
table_data = []

for row in rows:
    data = {}
    data['Query'] = row.find('td', {'data-bt': 'export_name'}).text
    data['Date'] = row.find('td', {'data-bt': 'table-column-date'}).text
    data['Source'] = row.find('td', {'data-bt': 'table-column-source'}).text
    data['Records'] = row.find('td', {'data-bt': 'table-column-records'}).text
    data['File size'] = row.find('td', {'data-bt': 'table-column-filesize'}).text
    data['Format'] = row.find('td', {'data-bt': 'table-column-format'}).text
    data['Download_Link'] = row.find('a', {'data-bt': 'download_export'})['href']
    
    table_data.append(data)

driver.quit()

# Data Processing to Create Filename, Publication_Title columns

In [5]:
# Convert list of dicts to DataFrame
df = pd.DataFrame(table_data)

# Create 'Filename' column
df['Filename'] = df['Download_Link'].apply(lambda x: x.split('/')[-1])

# Create 'Publication_Title' column
df['Publication_Title'] = df['Query'].apply(lambda x: x.split(':')[-1]).str.strip('"')

# Set the output file path
xlsx_file_path = '/Users/sachinsrivastava/Downloads/PythonCodes_Local/Hass_DataInnovationLab/DimensionsTask_06172023/Dimensions_ExportCenterData_output_06192023_v3.0.xlsx'

# Save DataFrame to Excel file
df.to_excel(xlsx_file_path, index=False)

# Download the Files

In [7]:
import requests
import os
import time

# Set the download directory
download_directory = '/Users/sachinsrivastava/Downloads/PythonCodes_Local/Hass_DataInnovationLab/Dimensions_Downloads'

# Ensure the download directory exists
os.makedirs(download_directory, exist_ok=True)

# Iterate over the download links
for index, row in df.iterrows():
    url = row['Download_Link']
    filename = row['Filename']

    # Create the full file path
    file_path = os.path.join(download_directory, filename)

    # Download the file
    response = requests.get(url, allow_redirects=True)
    
    # Write the content to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    
    # Wait for 1 second before next download
    time.sleep(1)

KeyboardInterrupt: 